# Optimize CNN model using keras-tuner

In [2]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 2.0MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=95cd7ffab290835c05b8709ccb28a4cbea490e6f0a7bc8dd87477fc219d89d60
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=e9095c6e4f5ca5994b4a70d2d55710f7b218e754bb97444c6287e6d8d44ef6c2
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [3]:
import tensorflow as tf
from tensorflow import keras 
import numpy as np

In [4]:
print(tf.__version__)

2.3.0


## **FASHION MNIST DATASET**

In [5]:
fashion_mnist=keras.datasets.fashion_mnist

In [6]:
(train_img,train_label),(test_img,test_label)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [7]:
train_img[0].shape

(28, 28)

**Scaling the data**

In [8]:
train_img=train_img/255.0
test_img=test_img/255.0

In [9]:
train_img[0].shape

(28, 28)

In [10]:
train_img=train_img.reshape(len(train_img),28,28,1)
test_img=test_img.reshape(len(test_img),28,28,1)

**Model creation**

In [11]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [12]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [13]:
tuner_search= RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output',project_name='Mnist fashion')

In [14]:
tuner_search.search(train_img, train_label,epochs=3,validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 6s 3ms/step - loss: 0.3855 - accuracy: 0.8613 - val_loss: 0.3047 - val_accuracy: 0.8863
Epoch 2/3
1688/1688 [==============================] - 5s 3ms/step - loss: 0.2425 - accuracy: 0.9091 - val_loss: 0.2671 - val_accuracy: 0.9038
Epoch 3/3
1688/1688 [==============================] - 5s 3ms/step - loss: 0.1818 - accuracy: 0.9319 - val_loss: 0.2593 - val_accuracy: 0.9043


Epoch 1/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.4480 - accuracy: 0.8423 - val_loss: 0.4204 - val_accuracy: 0.8468
Epoch 2/3
1688/1688 [==============================] - 6s 3ms/step - loss: 0.3352 - accuracy: 0.8765 - val_loss: 0.3816 - val_accuracy: 0.8578
Epoch 3/3
1688/1688 [==============================] - 6s 3ms/step - loss: 0.3023 - accuracy: 0.8896 - val_loss: 0.3278 - val_accuracy: 0.8795


Epoch 1/3
1688/1688 [==============================] - 6s 3ms/step - loss: 0.4272 - accuracy: 0.8470 - val_loss: 0.2997 - val_accuracy: 0.8918
Epoch 2/3
1688/1688 [==============================] - 6s 3ms/step - loss: 0.2822 - accuracy: 0.8966 - val_loss: 0.3010 - val_accuracy: 0.8857
Epoch 3/3
1688/1688 [==============================] - 6s 3ms/step - loss: 0.2374 - accuracy: 0.9130 - val_loss: 0.2727 - val_accuracy: 0.8977


Epoch 1/3
1688/1688 [==============================] - 6s 3ms/step - loss: 0.5343 - accuracy: 0.8137 - val_loss: 0.4207 - val_accuracy: 0.8392
Epoch 2/3
1688/1688 [==============================] - 6s 3ms/step - loss: 0.3951 - accuracy: 0.8546 - val_loss: 0.3972 - val_accuracy: 0.8578
Epoch 3/3
1688/1688 [==============================] - 6s 3ms/step - loss: 0.3746 - accuracy: 0.8611 - val_loss: 0.4086 - val_accuracy: 0.8538


Epoch 1/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.4588 - accuracy: 0.8362 - val_loss: 0.3567 - val_accuracy: 0.8697
Epoch 2/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.3336 - accuracy: 0.8776 - val_loss: 0.3616 - val_accuracy: 0.8773
Epoch 3/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.2934 - accuracy: 0.8908 - val_loss: 0.3280 - val_accuracy: 0.8770


INFO:tensorflow:Oracle triggered exit


**Select the best model**

In [17]:
model=tuner_search.get_best_models(num_models=1)[0]

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 32)        25632     
_________________________________________________________________
flatten (Flatten)            (None, 15488)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                1486944   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 1,513,866
Trainable params: 1,513,866
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.fit(train_img,train_label,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.1336 - accuracy: 0.9502 - val_loss: 0.2709 - val_accuracy: 0.9143
Epoch 5/10
1688/1688 [==============================] - 5s 3ms/step - loss: 0.0939 - accuracy: 0.9648 - val_loss: 0.3423 - val_accuracy: 0.9098
Epoch 6/10
1688/1688 [==============================] - 5s 3ms/step - loss: 0.0653 - accuracy: 0.9760 - val_loss: 0.3693 - val_accuracy: 0.9160
Epoch 7/10
1688/1688 [==============================] - 5s 3ms/step - loss: 0.0501 - accuracy: 0.9823 - val_loss: 0.3907 - val_accuracy: 0.9113
Epoch 8/10
1688/1688 [==============================] - 5s 3ms/step - loss: 0.0363 - accuracy: 0.9871 - val_loss: 0.4283 - val_accuracy: 0.9077
Epoch 9/10
1688/1688 [==============================] - 5s 3ms/step - loss: 0.0295 - accuracy: 0.9889 - val_loss: 0.4688 - val_accuracy: 0.9133
Epoch 10/10
1688/1688 [==============================] - 5s 3ms/step - loss: 0.0243 - accuracy: 0.9916 - val_loss: 0.5154 - val_accuracy

**For the purpose of hyper parameter tuning , keras-tuner is a great library to find out the best parameters for the model.**